# Data Preparation

In [ ]:
import huggingface_hub

huggingface_hub.login()

In [ ]:
import wandb

wandb.login()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Kanisorn P\.netrc


In [3]:
# !mkdir "weights"

## Import Library

In [4]:
%%capture
# !pip install pythainlp datasets transformers

In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from tqdm.notebook import tqdm
import pythainlp
from IPython.display import display
from sklearn.utils import resample 

# Hyperparameters

In [6]:
model_names = ["clicknext/phayathaibert", "pythainlp/phayathai-bert-base"]

NUM_EPOCHS = 10
BATCH_SIZE = 8
MODEL_NAME = "clicknext/phayathaibert"
RANDOM_STATE = 49

# Load Data

In [7]:
dataset_path = '../dataset'
df_train = pd.read_csv(dataset_path + "/train.csv")
df_test = pd.read_csv(dataset_path + "/test.csv")

In [8]:
print("Train data shape: ", df_train.shape)
print("Test data shape: ", df_test.shape)
record_train = df_train.shape[0]

Train data shape:  (362, 5)
Test data shape:  (90, 4)


In [9]:
df_train.head()

,ID,set,question,answer,score
0,0,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรจะมีค่าต่ำ เพราะว่าเราต้องการทร...,0.0
1,1,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เพราะเป็นการเก็บข้อมูลจากหลาย users ...,5.0
2,2,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรเป็น #checkout events/ #cookies...,5.0
3,3,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,"เห็นด้วย ให้ X~Binomial(N,p) โดย p เป็นอัตราส่...",2.0
4,4,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เนื่องจากการทดสอบ A/B Testing เป็นวิธ...,4.5


In [10]:
df_train[['set', 'question', 'answer']].describe()

,set,question,answer
count,362,362,362
unique,4,4,362
top,Q4,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,ใช้ เพราะเป็นการใช้ scientific method มาช่วยหา...
freq,91,91,1


In [11]:
df_train[['score']].describe()

,score
count,362.000000
mean,3.021409
std,1.990644
min,0.000000
25%,1.000000
50%,3.500000
75%,5.000000
max,5.000000


In [12]:
df_test.head()

,ID,set,question,answer
0,362,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,ได้ เพราะ โจทย์นี้ ตัวแปรคือค่าเฉลี่ยของอัตราก...
1,363,Q4,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,50/50 มีข้อดีก็คือสามารถเก็บผลการทดสอบได้ไวกว่...
2,364,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,1.จำนวนการเข้าชม 2. จำนวนการคลิกโฆษณา 3.เวลาที...
3,365,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เพราะการคลิกมีผลลัพธ์ได้ 2 รูปแบบ นั่...
4,366,Q4,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,50/50 - จะใช้เวลาเก็บข้อมูลน้อยกว่า แต่มีความเ...


In [13]:
df_test[['set', 'question', 'answer']].describe()

,set,question,answer
count,90,90,90
unique,4,4,90
top,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,ได้ เพราะ โจทย์นี้ ตัวแปรคือค่าเฉลี่ยของอัตราก...
freq,23,23,1


# Reading Raw Data

## Questions

In [14]:
q = 'Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาตให้ผู้ใช้อัปโหลด แชร์ และดูวิดีโอได้ แฮมทาโร่ เป็นหัวหน้าทีมการตลาดของ Hamtube และเขาต้องการทราบว่าการย้ายตำแหน่งของโฆษณาจะช่วยเพิ่มยอดขาย (ผู้ใช้คลิกโฆษณามากขึ้น) หรือไม่ ดังนั้นเขาตัดสินใจที่จะดำเนินการทดลอง A/B testing. '
print(q)

Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาตให้ผู้ใช้อัปโหลด แชร์ และดูวิดีโอได้ แฮมทาโร่ เป็นหัวหน้าทีมการตลาดของ Hamtube และเขาต้องการทราบว่าการย้ายตำแหน่งของโฆษณาจะช่วยเพิ่มยอดขาย (ผู้ใช้คลิกโฆษณามากขึ้น) หรือไม่ ดังนั้นเขาตัดสินใจที่จะดำเนินการทดลอง A/B testing. 


In [15]:
for i in range(4):
    print(f"Q{i+1}. {df_train['question'].unique()[i][269:]}")
    print()


df_train[['set']].value_counts()

Q1. granularity ควรเป็นอะไร และความยาวของ attribution period ควรเป็นเท่าไหร่ จงอธิบายเหตุผล

Q2. แฮมทาโร่เลือกที่จะใช้ binomial distribution เพื่อแทน distribution ของการทำ A/B testing ในครั้งนี้ คุณเห็นด้วยหรือไม่เห็นด้วยกับแฮมทาโร่ เพราะเหตุใด

Q3. อธิบายเหตุผลที่แฮมทาโร่ควรใช้ A/B testing ในการทดสอบ หรือเสนอข้อโต้แย้งหากคุณไม่เห็นด้วย พร้อมกับอธิบายเหตุผล

Q4. แฮมทาโร่จะต้องเลือกว่าอยากให้สัดส่วนของ user ที่เห็นโฆษณาตำแหน่งเก่า ต่อ user ที่เห็นโฆษณาตำแหน่งใหม่เป็นเท่าไร โดยตอนนี้แฮมทาโร่กำลังลังเลระหว่างสัดส่วน 50/50 กับ สัดส่วน 80/20 จงอธิบายข้อดีข้อเสียของการเลือกสัดส่วนแต่ละแบบ และตอบว่าแบบใดที่น่าจะเหมาะสมกับปัญหานี้มากกว่า



set
Q1     91
Q4     91
Q2     90
Q3     90
Name: count, dtype: int64

In [16]:
df_test[['set']].value_counts()

set
Q2     23
Q3     23
Q1     22
Q4     22
Name: count, dtype: int64

In [17]:
df_train[['score']].value_counts()

score
5.00     150
1.00      58
0.00      50
3.00      26
2.00      20
4.00      15
0.50      10
4.50       9
3.50       8
1.50       8
2.50       5
0.75       1
4.25       1
4.75       1
Name: count, dtype: int64

In [18]:
for q in df_train["set"].unique():

    # Filter data for the current question
    df_q = df_train[df_train["set"] == q]
    print(f"Question: {q}")
    display(df_q[['score']].value_counts().sort_index(ascending=False))

Question: Q2


score
5.00     19
4.75      1
4.50      2
4.25      1
4.00      5
3.50      8
3.00      3
2.50      4
2.00      1
1.50      1
0.75      1
0.50      1
0.00     43
Name: count, dtype: int64

Question: Q3


score
5.0      66
3.0       1
2.0      10
1.0       9
0.0       4
Name: count, dtype: int64

Question: Q1


score
5.0       9
4.5       5
4.0       3
3.0      10
2.5       1
2.0       8
1.5       7
1.0      39
0.5       7
0.0       2
Name: count, dtype: int64

Question: Q4


score
5.0      56
4.5       2
4.0       7
3.0      12
2.0       1
1.0      10
0.5       2
0.0       1
Name: count, dtype: int64

# Clean Data

In [34]:
q = 2
sample = df_train[df_train['set'] == f'Q{q}'].sample(5)

print(df_train['question'].unique()[q-1][269:])
print()
    
for record in sample.iterrows():
    print(record[1]['score'])
    print(record[1]['answer'])
    print()

แฮมทาโร่เลือกที่จะใช้ binomial distribution เพื่อแทน distribution ของการทำ A/B testing ในครั้งนี้ คุณเห็นด้วยหรือไม่เห็นด้วยกับแฮมทาโร่ เพราะเหตุใด

0.0
User Level: การตั้งค่า granularity ที่ระดับผู้ใช้ (user level) สามารถให้ข้อมูลที่ละเอียดมากเกี่ยวกับพฤติกรรมของแต่ละบุคคล ทำให้ง่ายต่อการทำให้กลยุทธ์ตลาดเป็นไปตามความต้องการของกลุ่มเป้าหมาย

Channel Level: การรวบรวมข้อมูลที่ระดับช่องทาง (channel level) อาจช่วยให้ทราบถึงประสิทธิภาพของแต่ละช่องทางตลาด ทำให้สามารถปรับกลยุทธ์การตลาดในแต่ละช่องทางได้

Campaign Level: การตั้งค่า granularity ที่ระดับแคมเปญ (campaign level) จะให้ภาพรวมของประสิทธิภาพของแคมเปญต่าง ๆ และช่วยในการวิเคราะห์ผลลัพธ์ของกิจกรรมตลาดทั้งหมด
Attribution period ควรเป็นเท่าไหร่ขึ้นอยู่กับลักษณะธุรกิจและลักษณะการพฤติกรรมของลูกค้า. ถ้าธุรกิจมีลูกค้าที่ต้องการเวลาในการตัดสินใจนาน, attribution period ควรเป็นระยะยาวเพื่อครอบคลุมกระบวนการนี้. ส่วนถ้าลูกค้ามีรูปแบบการพฤติกรรมที่รวดเร็ว, attribution period สั้นก็อาจเพียงพอในการวิเคราะห์และปรับกลยุทธ์ตลาด. การกำหนดระยะเวลานี้ควรสอดค

In [ ]:
stopwords = list(pythainlp.corpus.common.thai_stopwords())
print(stopwords)

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords])

['เพื่อที่', 'วันนี้', 'รึ', 'ช้าๆ', 'บ่อย', 'ก็ได้', 'บางขณะ', 'จนแม้น', 'พอเพียง', 'มันๆ', 'ข้า', 'แก', 'แค่ว่า', 'บางคราว', 'คล้ายกันกับ', 'ที', 'ตลอดไป', 'ทั้งปวง', 'เมื่อคราว', 'กัน', 'กลุ่ม', 'อย่างใด', 'ที่', 'แต่ก่อน', 'เนี่ยเอง', 'ควร', 'ฝ่ายใด', 'สิ้นกาลนาน', 'ให้ดี', 'จนทั่ว', 'ไม่ใช่', 'เพื่อ', 'เรียก', 'หมด', 'เท่านี้', 'ค่อนข้าง', 'ตลอด', 'ทีๆ', 'นอกนั้น', 'ด้วยเพราะ', 'เช่นดัง', 'หมดกัน', 'บอกแล้ว', 'ช่วงนั้น', 'คล้ายกับ', 'ระหว่าง', 'ถ้าหาก', 'ได้แก่', 'เสร็จสมบูรณ์', 'บ้าง', 'จาก', 'สําหรับ', 'ใกล้ๆ', 'หลังจาก', 'คุณๆ', 'เป็นต้นมา', 'ครั้งหลังสุด', 'พร้อมกัน', 'จึงจะ', 'ฝ่าย', 'อดีต', 'จริง', 'ทั้ง', 'อย่างยิ่ง', 'แสดงว่า', 'เชื่อว่า', 'นับแต่นั้น', 'ร่วมกัน', 'รวมถึง', 'ๆ', 'ขอ', 'แก้ไข', 'ที่นั้น', 'ครั้งหนึ่ง', 'คราวที่', 'พอกัน', 'คราวหน้า', 'ก็แล้วแต่', 'ก็ตาม', 'ค่อนมาทาง', 'จัดหา', 'สั้นๆ', 'เช่นเดียวกับ', 'จ๊ะ', 'พอแล้ว', 'เชื่อมั่น', 'เชื่อ', 'จากนี้ไป', 'คล้ายกับว่า', 'ข้าพเจ้า', 'สุด', 'ด้วยกัน', 'พอสม', 'แต่อย่างใด', 'ใหม่', 'เรื่อยๆ', 'พร้อม', 'เช่นที่เคย'

In [21]:
"""
Copied from thai2transformers (https://github.com/vistec-AI/thai2transformers/blob/master/thai2transformers/preprocess.py)
"""
from typing import Collection, Callable
import re
from pythainlp.tokenize import word_tokenize

_TK_UNK, _TK_REP, _TK_WREP, _TK_URL, _TK_END = "<unk> <rep> <wrep> <url> </s>".split()

SPACE_SPECIAL_TOKEN = "<_>"

def rm_brackets(text: str) -> str:
    """
        Remove all empty brackets and artifacts within brackets from `text`.
        :param str text: text to remove useless brackets
        :return: text where all useless brackets are removed
        :rtype: str
        :Example:
            >>> rm_brackets("hey() whats[;] up{*&} man(hey)")
            hey whats up man(hey)
    """
    # remove empty brackets
    new_line = re.sub(r"\(\)", "", text)
    new_line = re.sub(r"\{\}", "", new_line)
    new_line = re.sub(r"\[\]", "", new_line)
    # brakets with only punctuations
    new_line = re.sub(r"\([^a-zA-Z0-9ก-๙]+\)", "", new_line)
    new_line = re.sub(r"\{[^a-zA-Z0-9ก-๙]+\}", "", new_line)
    new_line = re.sub(r"\[[^a-zA-Z0-9ก-๙]+\]", "", new_line)
    # artifiacts after (
    new_line = re.sub(r"(?<=\()[^a-zA-Z0-9ก-๙]+(?=[a-zA-Z0-9ก-๙])", "", new_line)
    new_line = re.sub(r"(?<=\{)[^a-zA-Z0-9ก-๙]+(?=[a-zA-Z0-9ก-๙])", "", new_line)
    new_line = re.sub(r"(?<=\[)[^a-zA-Z0-9ก-๙]+(?=[a-zA-Z0-9ก-๙])", "", new_line)
    # artifacts before )
    new_line = re.sub(r"(?<=[a-zA-Z0-9ก-๙])[^a-zA-Z0-9ก-๙]+(?=\))", "", new_line)
    new_line = re.sub(r"(?<=[a-zA-Z0-9ก-๙])[^a-zA-Z0-9ก-๙]+(?=\})", "", new_line)
    new_line = re.sub(r"(?<=[a-zA-Z0-9ก-๙])[^a-zA-Z0-9ก-๙]+(?=\])", "", new_line)
    return new_line

def replace_newlines(text: str) -> str:
    """
        Replace newlines in `text` with spaces.
        :param str text: text to replace all newlines with spaces
        :return: text where all newlines are replaced with spaces
        :rtype: str
        :Example:
            >>> rm_useless_spaces("hey whats\n\nup")
            hey whats  up
    """

    return re.sub(r"[\n]", " ", text.strip())

def rm_useless_spaces(text: str) -> str:
    """
        Remove multiple spaces in `text`. (code from `fastai`)
        :param str text: text to replace useless spaces
        :return: text where all spaces are reduced to one
        :rtype: str
        :Example:
            >>> rm_useless_spaces("oh         no")
            oh no
    """
    return re.sub(" {2,}", " ", text)

def replace_spaces(text: str, space_token: str = SPACE_SPECIAL_TOKEN) -> str:
    """
        Replace spaces with _
        :param str text: text to replace spaces
        :return: text where all spaces replaced with _
        :rtype: str
        :Example:
            >>> replace_spaces("oh no")
            oh_no
    """
    return re.sub(" ", space_token, text)

def replace_rep_after(text: str) -> str:
    """
    Replace repetitions at the character level in `text`
    :param str text: input text to replace character repetition
    :return: text with repetitive tokens removed.
    :rtype: str
    :Example:
        >>> text = "กาาาาาาา"
        >>> replace_rep_after(text)
        'กา'
    """

    def _replace_rep(m):
        c, cc = m.groups()
        return f"{c}"

    re_rep = re.compile(r"(\S)(\1{3,})")
    return re_rep.sub(_replace_rep, text)

def replace_wrep_post(toks: Collection[str]) -> Collection[str]:
    """
    Replace reptitive words post tokenization;
    fastai `replace_wrep` does not work well with Thai.
    :param Collection[str] toks: list of tokens
    :return: list of tokens where repetitive words are removed.
    :rtype: Collection[str]
    :Example:
        >>> toks = ["กา", "น้ำ", "น้ำ", "น้ำ", "น้ำ"]
        >>> replace_wrep_post(toks)
        ['กา', 'น้ำ']
    """
    previous_word = None
    rep_count = 0
    res = []
    for current_word in toks + [_TK_END]:
        if current_word == previous_word:
            rep_count += 1
        elif (current_word != previous_word) & (rep_count > 0):
            res += [previous_word]
            rep_count = 0
        else:
            res.append(previous_word)
        previous_word = current_word
    return res[1:]

def remove_space(toks: Collection[str]) -> Collection[str]:
    """
    Do not include space for bag-of-word models.
    :param Collection[str] toks: list of tokens
    :return: Collection of tokens where space tokens (" ") are filtered out
    :rtype: Collection[str]
    :Example:
        >>> toks = ['ฉัน','เดิน',' ','กลับ','บ้าน']
        >>> remove_space(toks)
        ['ฉัน','เดิน','กลับ','บ้าน']
    """
    res = []
    for t in toks:
        t = t.strip()
        if t:
            res.append(t)
    return res

# combine them together
def process_transformers(
    text: str,
    pre_rules: Collection[Callable] = [
        rm_brackets,
        replace_newlines,
        rm_useless_spaces,
        replace_spaces,
        replace_rep_after,
    ],
    tok_func: Callable = word_tokenize,
    post_rules: Collection[Callable] = [replace_wrep_post],
) -> str:
    text = text.lower()
    for rule in pre_rules:
        text = rule(text)
    toks = tok_func(text)
    for rule in post_rules:
        toks = rule(toks)
    return "".join(toks)

In [32]:
def clean_text(text):
    text = text.apply(remove_stopwords) # remove unimportant words
    text = text.str.replace(r'\b\d+\.\s*', '', regex=True) # remove index numbers
    text = text.str.replace(r'[^\w\s\u0E01-\u0E5B/]', ' ', regex=True) 
    text = text.apply(process_transformers) # remove brackets, newlines, spaces, repetitions
    return text

In [35]:
test = "1. ทดสอบการลบตัวเลข (50/50)!?@# ออกจากข้อความ"
print(test)
print(clean_text(pd.Series([test]))[0])

1. ทดสอบการลบตัวเลข (50/50)!?@# ออกจากข้อความ
ทดสอบการลบตัวเลข<_>50/50<_>ออกจากข้อความ


In [ ]:
df_train['answer'] = clean_text(df_train['answer'])
df_test['answer'] = clean_text(df_test['answer'])

# Separate Data

In [ ]:
df_train_dq = df_train.drop(columns=['question'])
df_train_dq.head()

In [ ]:
df_q1 = df_train_dq[df_train_dq['set'] == "Q1"]
df_q2 = df_train_dq[df_train_dq['set'] == "Q2"]
df_q3 = df_train_dq[df_train_dq['set'] == "Q3"]
df_q4 = df_train_dq[df_train_dq['set'] == "Q4"]

display(df_q1.head())

# Train Model

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, ClassLabel
from sklearn.model_selection import KFold

In [ ]:
test_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# get 90 percentle of the length of the tokenized text
X = df_train["answer"].tolist()
lengths = [len(test_tokenizer.tokenize(text)) for text in X]
max_len = int(np.percentile(lengths, 95))
max_len

In [ ]:
MAX_LENGTH = 200

In [ ]:
# Load dataset
# Load PhayaThaiBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(
                MODEL_NAME,
                revision='main',
                max_length=MAX_LENGTH,
                truncation=True,
                return_tensors="pt"
                )

#create model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    revision='main',
    num_labels=12,
    problem_type="multi_label_classification"
)

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
from sklearn.metrics import mean_squared_error

# Define a function to compute MSE
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[0] if isinstance(predictions, tuple) else predictions
    mse = mean_squared_error(labels, predictions)
    return {"mse": mse}

In [ ]:
torch.cuda.empty_cache()  # Add this before/after running inference

In [ ]:
for q in df_train["set"].unique():
    print(f"Training model for {q}...")

    # Filter data for the current question
    df_q = df_train[df_train["set"] == q]

    # Prepare features and labels
    X = df_q["answer"].tolist()
    y = df_q["score"].tolist()
    bin_edges = [0, 1.5, 3.5, 5]
    bin_labels = [0, 1, 2,]
    df_q["score_bin"] = pd.cut(df_q["score"], bins=bin_edges, labels=bin_labels, include_lowest=True)
    
    # Tokenize input text
    def preprocess_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    # Convert data to Hugging Face Dataset format
    dataset = Dataset.from_dict({"text": X, "label": y, "score_bin" : df_q['score_bin'].tolist()})
    dataset = dataset.cast_column("score_bin", ClassLabel(num_classes=5))
    dataset = dataset.map(preprocess_function, batched=True)
    
    # Split into train and validation sets

    train_test = dataset.train_test_split(test_size=0.2, seed=RANDOM_STATE, stratify_by_column="score_bin")
    train_test["train"] = train_test["train"].remove_columns(["score_bin"])
    train_test["test"] = train_test["test"].remove_columns(["score_bin"])
    train_dataset = train_test["train"]
    val_dataset = train_test["test"]
    

    # Load PhayaThaiBERT model for regression (single output neuron)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f"weights/model_{q}",
        evaluation_strategy="steps",
        save_strategy="no",
        learning_rate=2e-5,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=NUM_EPOCHS,
        weight_decay=0.01,
        logging_dir=None,
        report_to="wandb", 
        logging_steps=record_train // (BATCH_SIZE * 10),  
    )


    # Define Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )
    
    wandb.init(project="NLP-midterm-2025", name=f"V0-{q}-b{BATCH_SIZE}-{NUM_EPOCHS}e")
    
    # Train the model
    trainer.train()

    # Save model and tokenizer
    model.save_pretrained(f"weights/model_{q}")
    tokenizer.save_pretrained(f"weights/model_{q}")
    print(f"Model for {q} saved successfully!\n")
    
    wandb.finish()


In [ ]:
def predict_answer(model_path, text):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = model(**inputs).logits.item()
    
    return output

In [ ]:
sample_record = df_train.sample(5)[0]
q = sample_record['set']
answer = sample_record['answer']
score = sample_record['score']

model_path = f"weights/model_{q}"

predicted_score = predict_answer(model_path, answer)
print(f"Predicted Score: {predicted_score}")
print(f"Expected Score :{score}")


In [ ]:
# Prepare output list
predictions = []

# Device configuration (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for _, row in df_test.iterrows():
    question = row["set"]
    answer = row["answer"]

    # Load the correct model for the question
    model_path = f"weights/model_{question}"  # Use best/final model
    model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
    model.eval()  # Set to evaluation mode

    # Tokenize the answer
    inputs = tokenizer(answer, return_tensors="pt", padding=True, truncation=True).to(device)

    # Predict score
    with torch.no_grad():
        output = model(**inputs).logits
        predicted_score = output.item()  # Convert tensor to scalar

    # Round and clip the score (to match 0-5 scale with 0.25 increments)
    rounded_score = torch.clamp(torch.round(torch.tensor(predicted_score) * 4) / 4, 0, 5).item()

    # Store result
    predictions.append({"ID": row["ID"], "score": rounded_score})

# Convert to DataFrame and save predictions
pred_df = pd.DataFrame(predictions)
pred_df.to_csv(f"predictions-BERT.csv", index=False)

print("Predictions saved!")


In [ ]:
pred_df.to_csv(f"predictions-BERT.csv", index=False)

In [ ]:
predictions = pd.read_csv(f"predictions-BERT.csv")

predictions.describe()

# Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
sample_df = df_train.sample(20)

predict_answers = []
ground_truths = []

for _, row in sample_df.iterrows():
    question = row["set"]
    answer = row["answer"]

    # Load the correct model for the question
    model_path = f"weights/model_{question}"  # Use best/final model
    model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
    model.eval()  # Set to evaluation mode

    # Tokenize the answer
    inputs = tokenizer(answer, return_tensors="pt", padding=True, truncation=True).to(device)

    # Predict score
    with torch.no_grad():
        output = model(**inputs).logits
        predicted_score = output.item()  # Convert tensor to scalar

    # Round and clip the score (to match 0-5 scale with 0.25 increments)
    rounded_score = torch.clamp(torch.round(torch.tensor(predicted_score) * 4) / 4, 0, 5).item()

    # Store result
    predict_answers.append({"score": rounded_score})
    ground_truths.append(row["score"])
    

print("MSE for sample data:", mean_squared_error(ground_truths, predict_answers))
